<center> <font size=6>Stochastic Optimization</font></center>
<center> <font size=4>PW4 Machine Replacement Problem</font></center>

# Introduction

In this problem, we want to optimize the moment when we are to replace a machinery in a candy factory.  
At the end of each production cycle (e.g. seasonal) a candy production line must decide whether to keep a machinery again or replace it with a new one.

# 1. Bellman Equation

_**Write down the Bellman optimality equation of the value function.**_

The Bellman optimality equation for this problem is given by:

$$V*(s) = max[ y(s)m + \gamma ∑T(s,a,s')V*(s') ]$$

where $V*(s)$ is the optimal value function for state $s$, $y(s)$ is the production efficiency of the machinery at state $s$, $m$ is the profit contribution of candy per ton, $\gamma$ is the discount factor, $T(s,a,s')$ is the state transition probability from state $s$ to state $s'$ under action $a$, and the summation is over all possible next states $s'$.

# 2. Value Iteration

_**What replacement policy maximizes the expected long term cumulative profits?**_   
_**Using Value Iteration to solve the problem (you are also encouraged to use the Policy Iteration and Linear Program- ming method). Test the sensitivity of the optimal policies to different problem parameters, e.g., p and c.**_

We first need to define the **environment** in which the problem takes place as a **Markov Decision Process**.

### Markov Decision Process Environment

**Variables**

We have the following variables that are the *core* of our subject:
- Actions Set $\mathcal{A}$
- States Set $\mathcal{S}$
- Probability $\mathcal{p}$


In [46]:
# ============================ VARIABLES ============================ #
actions = ["keep","replace"]
states = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
p = 0.3

**Parameters**

We also need to define all the **hyperparameters** of our MDP problem that will be used when **solving**:

In [47]:
# ============================ PARAMETERS ============================ #
max_iter = 10000  # Maximum number of iterations
delta = 1e-400  # Error tolerance
V = [0,0,0,0,0,0,0,0,0,0]  # Initialize values
pi = [None,None,None,None,None,None,None,None,None,"replace"]  # Initialize policy
gamma = 0.9  # discount factor
theta = 0.1  # threshold parameter that defines when the change in the value function is negligible (i.e. when we can stop process of Policy Evaluation)
m = 150 #PROFIT
c = 500 #COST

**Functions**

In order to simulate the ***dynamics*** inside our MDP, some functions are to be defined:
- `action_function`: defines the *potential* actions that can be taken when at state $s$
- `reward_function`: defines the $reward$ given when action $a$ is taken while in state $s$
- `state_transition_function`: in this ***stochastic*** model, defines the **probability** of transitioning from state $s$ to next state $s'$ when taking action $a$

In [48]:
# ============================ FUNCTIONS ============================ #

# Action Function #
def action_function(current_state):
    if current_state < 10:
        actions = ["keep", "replace"]
        return actions  # Action 'Keep

    elif current_state == 10:
        actions = ["replace"]
        return actions  # Action 'Replace'

# Reward Function #
def reward_function(state, action,s_next,m,c):
    if action == "keep":
        return (
            8 + state - 0.15 * state**2
        ) * m  # Action 'Keep

    elif action == "replace":
        return (
            (8 + state - 0.15 * state**2) * m
        ) - c  # Action 'Replace'

# State Transition Function #
def state_transition_function(current_state, action, probability):
    """
    Return all non-zero probability transitions for this action
        from this state, as a list of (state, probability) pairs
    
    => Pr[s'|s,a] = Pr[S_{t+1} = s_prime | S_t = current_state, A_t = action]
    """

    if (
        (action == "keep") 
        and (current_state <= 8)
    ):
        return [(current_state + 1,probability),(current_state + 2, 1 - probability)]

    if (action == "keep") and (current_state == 9):
        return [(10, 1)]

    if (action == "keep") and (current_state == 10):
        return [(0, 0)]
        
    if (action == "replace"):
        return [(1, 1)]
 

### Value Iteration Algorithm

In [49]:
# Start value iteration
for i in range(max_iter):
    max_diff = 0  # Initialize max difference
    V_new = [0,0,0,0,0,0,0,0,0,0]  # Initialize values
    for s in states:
        max_val = 0
        for a in actions:
            val = 0.0
            # Bellman update
            for (s_next, transition_probability) in state_transition_function(s, a, p):
                reward = reward_function(s, a, s_next,m,c)  # Get direct reward
                val += transition_probability * (reward+ (gamma* V[s_next-1]))
                
            # Store value best action so far
            max_val = max(max_val, val)
            
            # Update best policy
            if V[s-1] < val:
                pi[s-1] = a # Store action with highest value

        V_new[s-1] = max_val  # Update value with highest value
        # Update maximum difference
        max_diff = max(max_diff, abs(V[s-1] - V_new[s-1]))

    # Update value functions
    V = V_new
   
    # If diff smaller than threshold delta for all states, algorithm terminates
    if max_diff < delta:
        break

In [50]:
for index, val in enumerate(V):
    print(f"Optimal Value Function V^* for state {index+1}: {val}")

Optimal Value Function V^* for state 1: 12589.879168355506
Optimal Value Function V^* for state 2: 12577.650209007583
Optimal Value Function V^* for state 3: 12486.37081257692
Optimal Value Function V^* for state 4: 12375.12712636796
Optimal Value Function V^* for state 5: 12218.391251519955
Optimal Value Function V^* for state 6: 12120.891251519955
Optimal Value Function V^* for state 7: 11978.391251519955
Optimal Value Function V^* for state 8: 11790.891251519955
Optimal Value Function V^* for state 9: 11558.391251519955
Optimal Value Function V^* for state 10: 11280.891251519955


In [51]:
for index, val in enumerate(pi):
    print(f"Optimal policy 𝜋^* for state {index+1}: {val}")

Optimal policy 𝜋^* for state 1: keep
Optimal policy 𝜋^* for state 2: keep
Optimal policy 𝜋^* for state 3: keep
Optimal policy 𝜋^* for state 4: keep
Optimal policy 𝜋^* for state 5: replace
Optimal policy 𝜋^* for state 6: replace
Optimal policy 𝜋^* for state 7: replace
Optimal policy 𝜋^* for state 8: replace
Optimal policy 𝜋^* for state 9: replace
Optimal policy 𝜋^* for state 10: replace


### Conclusion

In order to **maximize** profit, the candy factory should consider *replacing* only starting from state 6.

# 3. Policy Iteration

The optimal replacement policy that maximizes the expected long term cumulative profits is to replace the machinery at state 9 (i.e. the last state before becoming too unproductive) and keep it at all other states. This policy is derived by solving the Bellman optimality equation using the policy iteration algorithm.

![image](./val_iter_vs_pol_iter.png)

### Variables, Parameters & Functions

In the following code, we again define the states, actions, and hyperparams for our **Policy Iteration** problem for more consistency.

In [9]:
# Define the set of states S
S = list(range(1, 11))

# Define the problem parameters
p = 0.9
m = 150
c = 500
gamma = 0.9
epsilon = 1e-6

# Initialize the policy
pi = {}
for s in range(1, 10):
    pi[s] = {"keep":1,"replace":0}
    pi[10] = {"keep":0,"replace":1}

#Initialize the value function

V = {}
for s in range(1, 11):
    V[s] = 0

# Initialize the previous policy
old_pi = {}

# Define the production efficiency function
def y(s):
    return 8 + s - 0.15 * s**2

# Define the state transition probability function
def state_transition_probability(s, a, s_next):
    if a == 'keep':
        if s_next == s + 1 and s <= 8:
            return p
        elif s_next == s + 2 and s <= 8:
            return 1 - p
        elif s_next == 10 and s == 9:
            return 1
        else:
            return 0
    elif a == 'replace':
        if s_next == 1:
            return 1
        else:
            return 0

# Define the reward function
def reward_function(s, a):
    if a == 'keep':
        return y(s) * m
    elif a == 'replace':
        return y(s) * m - c

In [10]:
def improve_policy(pi, current_state, best_actions, p):
    """
    Defines a new policy pi(a|s) given the new best actions (computed by the Policy improvement)

    Args:
        pi (array): numpy array representing the policy
        current_state (int): Current state s_t of agent
        best_actions (list): list with best actions
        actions (list): list of every possible action (given by board.actions)
    """

    possible_actions = action_function(current_state)

    for a in possible_actions:
        for (s_next,proba) in state_transition_function(s, a, p):
            pi[current_state][a] = proba if a in best_actions else 0

    return pi[current_state]

In [1]:
def bellman_update(actions, V, old_v, state, pi, gamma,p):
    """
    Applies the Bellman update rule to the value function.

    Args:
        - actions (array): array of actions
        - V (array): numpy array representing the value function
        - old_v (array): numpy array representing the value function on the last iteration
        - state (int):  current state
        - pi (array):  array of current optimal policy
        - gamma (float): gamma parameter (between 0 and 1)
    """

    val = 0
    for a in actions:
        for (s_next, transition_probability) in state_transition_function(state, a, p):
            reward = reward_function(state, a)  # Get direct reward
            print(f"Next state:{s_next} | reward: {reward} | old_v[s_next] {old_v[s_next]}")
            val += pi[state][a] * (reward + (gamma * old_v[s_next]))

    # UPDATE OF the VALUE function
    V[state] = val
    return V

In [12]:
def policy_evaluation(states,actions, V, pi, gamma, theta,p):
    """
    Applies the policy evaluation algorithm.

    Args:
        states (array): array of states
        actions (array): array of actions
        V (array): numpy array representing the value function
        pi (array): numpy array representing the policy
        gamma (float): gamma parameter (between 0 and 1)
        theta (float): threshold parameter that defines when the change in the value function is negligible
    """

    delta = theta + 1
    iter = 0

    print(f"Delta: {delta} | Theta: {theta}")
    
    while delta >= theta:
        old_v = V.copy()
        delta = 0

        # Iterate all states
        for state in states:  # [1,...,10]
            # Run one iteration of the Bellman update rule for the value function
            V = bellman_update(actions, V, old_v, state, pi, gamma,p)
            # Compute difference for EACH STATE, and take the maximum difference
            delta = max(delta, abs(old_v[state] - V[state]))
        iter += 1

    print(
        f"\nValue function updated: the Policy Evaluation algorithm converged after {iter} sweeps"
    )

    return V

In [13]:
def policy_improvement(states,actions,V, pi, gamma,p):
    """
    Applies the Policy Improvement step.

    Args:
        board (Environment): gridworld environment
        v (array): numpy array representing the value function
        pi (array): numpy array representing the policy
        gamma (float): gamma parameter (between 0 and 1)
    """
    policy_stable = True

    # Iterate all states
    for s in states:  # [1,...,10]
        old_pi = pi[s].copy()

        # Instanciate best actions list & best action val
        best_actions = []
        max_action_val = None

        ############ COMPUTE the ACTION-value function Q_𝜋(s,a) for each action ############
        for a in actions:
            curr_action_value = 0.0
            # Bellman update
            for (s_next, transition_probability) in state_transition_function(s, a, p):
                reward = reward_function(s, a)  # Get direct reward
                curr_action_value += transition_probability * (reward+ (gamma* V[s_next]))

            if max_action_val is None:  # If no best action, add this one
                max_action_val = curr_action_value
                best_actions.append(a)
            elif (
                curr_action_value > max_action_val
            ):  # If better than precedent action, replace
                max_action_val = curr_action_value
                best_actions = [a]
            elif (
                curr_action_value == max_action_val
            ):  # If the Action-value for this specific actions equals another Action-value of another action, both deserve to be taken
                best_actions.append(a)

        # Define new policy pi(a|s) with the following variables
        # - pi: current policy, will be updated by new_policy
        # - current_state: state for which the policy should be updated
        # - new_policy: best actions to take for this specific state
        # - actions: all set of actions
        
        pi[s] = improve_policy(
            pi, s, best_actions, p
        )

        # Check whether the policy has changed
        if (old_pi == pi[s]):
            policy_stable = True

    if not policy_stable:
        print(f"\nPolicy improved for all states.")
    else:
        # Refresh the display
        print(f"\nPolicy is now STABLE !")

    return policy_stable

In [14]:
def policy_iteration(V,pi,states,actions,gamma,theta,p):
    """
    Runs the Policy Iteration algorithm:
        - Policy Evaluation
        - Policy Improvement

    Args:
        env (Environment): Environment class
        v0_val (int): initial value for the value function
        gamma (float): gamma parameter (between 0 and 1)
        theta (float): threshold parameter that defines when the change in the value function is negligible (i.e. when we can stop process)
        seed (int): seed (for matter of reproducible results)
    """
    timesteps = 0

    # Initialize policy as a NOT STABLE one
    policy_stable = False

    while True:
        if not policy_stable:
            timesteps += 1
            print(f"\nIteration {timesteps} of Policy Iteration algorithm")

            # ============== Policy Evaluation Step ============== #
            print("# ============== Policy Evaluation Step ============== #")
            V = policy_evaluation(states,actions, V, pi, gamma, theta)

            # ============== Policy Improvement Step ============== #
            print("# ============== Policy Improvement Step ============== #")
            policy_stable = policy_improvement(states,actions,V, pi, gamma,p)

            print(
                f"\The whole Policy Iteration (eval -> improvement -> eval -> ...) algorithm converged after {timesteps} steps"
            )

In [ ]:
policy_iteration(V,pi,S,actions,gamma,theta,p)

### Conclusion

We tried to implement this time the **Policy Iteration Algorithm**, however it seems that our algo runs indefinitely trying to make the *policy evaluation* converge to a treshold it cannot reach...

# 4. Varying parameters

To test the sensitivity of the optimal policies to different problem parameters, we can vary the values of $p$ and $c$ in the **Value Iteration Algorithm** and observe how the resulting policies and value functions change.  
- For example, if we increase the probability $p$ of transitioning to a higher efficiency state, we expect the optimal policy to become **more aggressive** (i.e. more likely to replace the machinery) since the machinery will go to higher states and be used **faster**.
- Similarly, if we increase the cost $c$ of replacing the machinery, we expect the optimal policy to become **more conservatibe** (i.e. less likely to replace the machinery) since the cost of replacing the machinery will become higher.

- $p = 0.3$ | $c = 500$

|State| $V^*$| $\pi^*$  |
|---|---|---------------|
| 1 |  12589.88 |Optimal policy $\pi^*$ for state 1: keep  
| 2  |  12577.65 |Optimal policy $\pi^*$ for state 2: keep  
| 3  |  12486.37 |Optimal policy $\pi^*$ for state 3: keep  
| 4  |  12375.13 |Optimal policy $\pi^*$ for state 4: keep  
|  5 | 12218.39|Optimal policy $\pi^*$ for state 5: replace  
|  6 |  12120.89  |Optimal policy $\pi^*$ for state 6: replace  
|  7 |  11978.39 |Optimal policy $\pi^*$ for state 7: replace  
|  8 |  11790.89 |Optimal policy $\pi^*$ for state 8: replace  
| 9  |  11558.39 |Optimal policy $\pi^*$ for state 9: replace   
| 10  | 11280.89  |Optimal policy $\pi^*$ for state 10: replace|




- $p = 0.9$ | $c = 3000$

|State| $V^*$| $\pi^*$  |
|---|---|---------------|
| 1 |  10390.13 |Optimal policy $\pi^*$ for state 1: keep  
| 2  |  10109.16 |Optimal policy $\pi^*$ for state 2: keep  
| 3  |  9713.43 |Optimal policy $\pi^*$ for state 3: keep  
| 4  |  9236.49 |Optimal policy $\pi^*$ for state 4: keep  
|  5 |  8715.26 |Optimal policy $\pi^*$ for state 5: keep  
|  6 |  8190.33 |Optimal policy $\pi^*$ for state 6: keep  
|  7 |  7706.58 |Optimal policy $\pi^*$ for state 7: keep  
|  8 |  7311.12 |Optimal policy $\pi^*$ for state 8: replace  
| 9  |  7078.62 |Optimal policy $\pi^*$ for state 9: replace   
| 10  | 6801.12  |Optimal policy $\pi^*$ for state 10: replace|